In [12]:
# 처음 잡은 object를 놓치지 않고, 끝까지 잡도록 유지하는 코드

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!nvidia-smi
import torch, sys, platform
print("Py:", sys.version)
print("Plat:", platform.platform())

Thu Sep 18 14:37:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             51W /  400W |   38211MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [14]:
# 0) YOLO 설치
!pip -q install ultralytics==8.3.197 lap>=0.5.12

from ultralytics import YOLO
PLAYER_MODEL = "/content/drive/MyDrive/Little_kid_0912/pose/player/player_detect/weights/best.pt"  # 네 가중치
player_model = YOLO(PLAYER_MODEL)

In [1]:
!pip install hydra-core==1.3.2 omegaconf==2.3.0

!pip install --upgrade torchvision

!pip install decord

In [2]:
# ================================
# ✅ SAM2 패키지 경로 등록 & 설치
# ================================
import sys, os

# SAM2가 Drive에 저장돼 있다고 가정
SAM2_PATH = "/content/drive/MyDrive/sam2"
sys.path.append(SAM2_PATH)

# 패키지 설치 (Colab 런타임마다 필요)
!pip install -e /content/drive/MyDrive/sam2

# ================================
# ✅ 라이브러리 import
# ================================
import torch, os, cv2, numpy as np
from collections import deque  # ★ NEW
from ultralytics import YOLO
from sam2.build_sam import build_sam2_video_predictor

# ------------------------
# 준비
# ------------------------
def best_dtype():
    return torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16
dtype = best_dtype()

# 성능 스위치 (가능한 경우) ★ NEW
torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision("high")
except Exception:
    pass

os.chdir("/content/drive/MyDrive/sam2")
ckpt = "/content/drive/MyDrive/sam2/checkpoints/sam2.1_hiera_large.pt"
cfg  = "configs/sam2.1/sam2.1_hiera_l.yaml"
pred = build_sam2_video_predictor(cfg, ckpt, vos_optimized=False)

MP4 = "/content/drive/MyDrive/Little_kid_0912/Video_data/0710_night2.mp4"
with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
    state = pred.init_state(MP4)

PLAYER_MODEL = "/content/drive/MyDrive/Little_kid_0912/pose/player/player_detect/weights/best.pt"
BALL_MODEL   = "/content/drive/MyDrive/Little_kid_0912/pose/ball/ball_800/train/weights/best.pt"
player_model = YOLO(PLAYER_MODEL)
ball_model   = YOLO(BALL_MODEL)

# 가능한 경우 모델 fuse 및 CUDA 이동 ★ NEW
try:
    player_model.fuse()
    ball_model.fuse()
except Exception:
    pass
player_model.to("cuda")
ball_model.to("cuda")

# ------------------------
# Util 함수
# ------------------------
def iou(boxA, boxB):
    """float 좌표에 맞춘 IOU ( +1 제거 )  ★ NEW"""
    xA = max(boxA[0], boxB[0]); yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2]); yB = min(boxA[3], boxB[3])
    inter = max(0.0, xB - xA) * max(0.0, yB - yA)
    areaA = max(0.0, (boxA[2] - boxA[0])) * max(0.0, (boxA[3] - boxA[1]))
    areaB = max(0.0, (boxB[2] - boxB[0])) * max(0.0, (boxB[3] - boxB[1]))
    denom = areaA + areaB - inter + 1e-6
    return inter / denom

def box_center(box):
    x1,y1,x2,y2 = box
    return (0.5*(x1+x2), 0.5*(y1+y2))

id2color, ema_masks, class_of = {}, {}, {}
def color_for(oid: int):
    if oid not in id2color:
        rng = np.random.default_rng(oid + 12345)
        id2color[oid] = tuple(int(c) for c in rng.integers(60,255,size=3))
    return id2color[oid]

# ------------------------
# 초기 씨딩 (각 객체가 '처음' 보이는 프레임에서)
# ------------------------
dev = torch.device("cuda")
next_oid = 0
class_of = {}   # {oid: "player"|"ball"}
found_players = False
found_ball = False

# 스캔 설정
SCAN_MAX = 300          # 처음 N프레임까지만 스캔 (원하면 크게)
STEP     = 1            # 1프레임 단위로 스캔 (속도 필요시 2~3으로)
CONF_P   = 0.30         # 선수 conf
CONF_B   = 0.04         # 공 conf(더 낮춤) ★ NEW
IMGSZ_B  = 1536         # 공 입력 해상도 상향(감지력↑) ★ NEW

cap = cv2.VideoCapture(MP4)

def seed_object(fi, xyxy, label, use_box=True):
    """
    xyxy=[x1,y1,x2,y2]를 받아 SAM2에 씨딩.
    - use_box=True: box 프롬프트 + 중심점(point) 함께 전달(권장)
    - use_box=False: 중심점(point)만 전달
    """
    global next_oid
    x1, y1, x2, y2 = xyxy
    oid = next_oid

    with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
        if use_box:
            # ✅ 박스 + 포인트(같은 CUDA)
            box_t = torch.tensor([[x1, y1, x2, y2]], device=dev, dtype=torch.float32)
            xc, yc = (x1 + x2) / 2.0, (y1 + y2) / 2.0
            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],        device=dev, dtype=torch.long)
            pred.add_new_points_or_box(
                state, fi, oid,
                points=pts, labels=labs,   # ← 포인트/라벨 명시
                box=box_t,                 # ← 박스도 함께
                normalize_coords=True
            )
        else:
            # 포인트만
            xc, yc = (x1 + x2) / 2.0, (y1 + y2) / 2.0
            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],        device=dev, dtype=torch.long)
            pred.add_new_points_or_box(
                state, fi, oid,
                points=pts, labels=labs,
                normalize_coords=True
            )

    class_of[oid] = label
    next_oid += 1
    return oid

seed_logs = []  # [(oid, label, fi)]

for fi in range(0, SCAN_MAX, STEP):
    ok, bgr = cap.read()
    if not ok:
        break

    # 초기 스캔: 선수 상위 N명만 씨딩(메모리/연산 낭비 방지) ★ NEW
    if not found_players:
        rp = player_model(bgr, conf=CONF_P, verbose=False)[0]
        if rp.boxes is not None and len(rp.boxes) > 0:
            confs = rp.boxes.conf.detach().float().cpu().numpy()
            order = np.argsort(-confs)[:10]  # 상위 10명만
            for idx in order:
                b = rp.boxes.xyxy[idx].detach().cpu().numpy().tolist()
                oid = seed_object(fi, b, "player", use_box=True)
                seed_logs.append((oid, "player", fi))
            found_players = True

    if not found_ball:
        # 공: 낮은 conf, 큰 imgsz, agnostic_nms 사용 ★ NEW
        rb = ball_model(bgr, conf=CONF_B, iou=0.4, imgsz=IMGSZ_B,
                        agnostic_nms=True, verbose=False)[0]
        if rb.boxes is not None and len(rb.boxes) > 0:
            # top-1이 아니라 가장 큰 bbox or conf 조합도 가능하지만 우선 conf 최대
            idx = int(rb.boxes.conf.argmax().detach().cpu())
            b = rb.boxes.xyxy[idx].detach().cpu().numpy().tolist()
            oid = seed_object(fi, b, "ball", use_box=True)
            seed_logs.append((oid, "ball", fi))
            found_ball = True

    # 둘 다 찾았으면 조기 종료
    if found_players and found_ball:
        break

cap.release()

# 요약 로그
n_players = sum(1 for _,lbl,_ in seed_logs if lbl=="player")
n_balls   = sum(1 for _,lbl,_ in seed_logs if lbl=="ball")
print("초기 씨딩 완료:",
      f"players={n_players}, ball={n_balls},",
      "details=", seed_logs)

def seed_from_yolo(res, label, fi=0):
    """현재는 사용 안 하지만 남겨둠"""
    global next_oid
    added = []
    if res.boxes is not None and len(res.boxes)>0:
        for b in res.boxes.xyxy.detach().cpu().numpy().tolist():
            x1,y1,x2,y2 = b
            xc,yc = (x1+x2)/2,(y1+y2)/2
            pts  = torch.tensor([[xc,yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],      device=dev, dtype=torch.long)
            oid  = next_oid
            with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
                pred.add_new_points_or_box(
                    state, fi, oid,
                    points=pts, labels=labs,
                    normalize_coords=True
                )
            class_of[oid] = label
            added.append((oid,[x1,y1,x2,y2]))
            next_oid += 1
    return added

# ------------------------
# 비디오 루프
# ------------------------
OUT_PATH = "/content/drive/MyDrive/Little_kid_0912/result/third_output.mp4"
cap = cv2.VideoCapture(MP4)
W,H,fps = int(cap.get(3)),int(cap.get(4)),cap.get(5) or 30
out = cv2.VideoWriter(OUT_PATH, cv2.VideoWriter_fourcc(*"mp4v"), fps,(W,H))

# 동적 리시드 주기를 위한 공 속도 추정 히스토리 ★ NEW
ball_hist = deque(maxlen=8)  # (fi, cx, cy)
def px_per_frame_speed():
    if len(ball_hist) < 2:
        return 0.0
    (f1, x1, y1), (f2, x2, y2) = ball_hist[-2], ball_hist[-1]
    dt = max(1, f2 - f1)
    return ((x2 - x1)**2 + (y2 - y1)**2)**0.5 / dt

max_jump_ppf = 50.0  # 프레임당 최대 허용 이동 픽셀(해상도/카메라에 맞게 조정) ★ NEW

# 신규 oid 발급 예산(프레임당) ★ NEW
NEW_OID_BUDGET_MAX = 3

with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
    for fi, obj_ids, masks in pred.propagate_in_video(state):
        # 프레임 동기화(안전) ★ NEW
        cap.set(cv2.CAP_PROP_POS_FRAMES, fi)
        ok, bgr = cap.read()
        if not ok: break
        overlay = bgr.copy()

        # ---------------- reseed ----------------
        # 공 속도 기반 동적 주기 ★ NEW
        speed = px_per_frame_speed()
        reseed_interval = 5 if speed > 25 else 15

        if fi % reseed_interval == 0:
            res_p = player_model(bgr, conf=0.3, verbose=False)[0]
            # 공: 낮은 conf / 큰 imgsz / agnostic_nms ★ NEW
            res_b = ball_model(bgr, conf=0.08, iou=0.4, imgsz=IMGSZ_B,
                               agnostic_nms=True, verbose=False)[0]

            new_oid_budget = NEW_OID_BUDGET_MAX  # 프레임당 신규 제한 ★ NEW

            for res, label in [(res_p, "player"), (res_b, "ball")]:
                if res.boxes is None or len(res.boxes) == 0:
                    continue

                # 후보들 순회(공일 때는 conf 높은 순으로 먼저 본다) ★ NEW
                confs = res.boxes.conf.detach().float().cpu().numpy()
                order = np.argsort(-confs)
                xyxys = res.boxes.xyxy.detach().cpu().numpy()

                for idx in order:
                    b = xyxys[idx].tolist()
                    matched = False

                    # 이전 EMA 마스크에서 박스 만들고 IOU & 크기변화율로 동일 객체 판단 ★ NEW
                    for oid in list(class_of.keys()):
                        if class_of[oid] != label:
                            continue
                        prev_mask = ema_masks.get(oid)
                        if prev_mask is None:
                            continue

                        ys, xs = np.where(prev_mask > 0.5)
                        if xs.size == 0:
                            continue

                        box_prev = [xs.min(), ys.min(), xs.max(), ys.max()]

                        # 크기 변화율 검사 ★ NEW
                        def ok_size(prev_box, bb, tol=3.0):
                            pw = max(1.0, prev_box[2]-prev_box[0]); ph = max(1.0, prev_box[3]-prev_box[1])
                            bw = max(1.0, bb[2]-bb[0]);            bh = max(1.0, bb[3]-bb[1])
                            s = (bw*bh)/(pw*ph)
                            return (1/tol) <= s <= tol

                        if iou(box_prev, b) > 0.5 and ok_size(box_prev, b, tol=3.0):
                            # ✅ 같은 객체로 판단 → 박스 + 포인트 프롬프트로 보강
                            x1, y1, x2, y2 = b
                            box_t = torch.tensor([[x1, y1, x2, y2]], device=dev, dtype=torch.float32)
                            xc, yc = (x1 + x2) / 2.0, (y1 + y2) / 2.0
                            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
                            labs = torch.tensor([1],        device=dev, dtype=torch.long)

                            pred.add_new_points_or_box(
                                state, fi, oid,
                                points=pts, labels=labs, box=box_t,
                                normalize_coords=True
                            )
                            matched = True

                            # 공 속도 히스토리 갱신 ★ NEW
                            if label == "ball":
                                ball_hist.append((fi, xc, yc))
                            break

                    if not matched:
                        # 신규 oid 발급 전, 기존 트랙들과 IOU가 너무 작으면 skip ★ NEW
                        if new_oid_budget <= 0:
                            continue

                        same_label_prev_boxes = []
                        for _oid, _lbl in class_of.items():
                            if _lbl != label: continue
                            pm = ema_masks.get(_oid)
                            if pm is None: continue
                            ys, xs = np.where(pm > 0.5)
                            if xs.size == 0: continue
                            same_label_prev_boxes.append([xs.min(), ys.min(), xs.max(), ys.max()])

                        max_prev_iou = 0.0
                        for pb in same_label_prev_boxes:
                            max_prev_iou = max(max_prev_iou, iou(pb, b))

                        if label == "ball":
                            min_iou_for_new = 0.05  # 작은 객체라 낮게
                        else:
                            min_iou_for_new = 0.10

                        if len(same_label_prev_boxes) > 0 and max_prev_iou < min_iou_for_new:
                            continue  # 노이즈 신규 방지

                        new_oid = seed_object(fi, b, label, use_box=True)
                        seed_logs.append((new_oid, label, fi))
                        new_oid_budget -= 1

                        # 공 신규 생성 시 히스토리에 추가 ★ NEW
                        if label == "ball":
                            cx, cy = box_center(b)
                            ball_hist.append((fi, cx, cy))

        # ---------------- 마스크 시각화 ----------------
        seen_ball_this_frame = False  # ★ NEW

        for oid, m in zip(obj_ids, masks):
            oid = int(oid)
            m = m.detach().cpu().numpy().squeeze()
            if m.ndim != 2:
                continue

            m = (m > 0).astype(np.uint8) * 255

            if class_of.get(oid) == "ball":
                # 공은 후처리 약하게(보존) ★ NEW
                m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((2,2), np.uint8))
                # blur는 생략 또는 아주 약하게
                seen_ball_this_frame = True
            else:
                m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))
                m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, np.ones((3,3), np.uint8))
                m = cv2.GaussianBlur(m, (3,3), 0)

            cur = (m > 127).astype(np.float32)
            prev = ema_masks.get(oid)
            keep = 0.35 if class_of.get(oid) == "ball" else 0.7  # 공은 반응성↑ ★ NEW
            ema = cur if prev is None else keep * prev + (1 - keep) * cur
            ema_masks[oid] = ema
            m_bin = ema > 0.5

            col = (0,0,255) if class_of.get(oid) == "ball" else color_for(oid)
            overlay[m_bin] = col
            ys, xs = np.where(m_bin)
            if xs.size:
                x1, x2, y1, y2 = xs.min(), xs.max(), ys.min(), ys.max()
                cv2.rectangle(overlay, (x1, y1), (x2, y2), col, 2)
                cv2.putText(overlay, f"{class_of.get(oid)} {oid}", (x1, max(0, y1-6)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, col, 2)

                # 공 중심 히스토리(마스크 기준)도 갱신 ★ NEW
                if class_of.get(oid) == "ball":
                    cx, cy = (x1 + x2)/2.0, (y1 + y2)/2.0
                    ball_hist.append((fi, cx, cy))

        bgr = cv2.addWeighted(overlay, 0.35, bgr, 0.65, 0)
        out.write(bgr)

cap.release(); out.release()
print("✅ saved:", OUT_PATH)

Obtaining file:///content/drive/MyDrive/sam2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=sam_2-1.0-0.editable-cp312-cp312-linux_x86_64.whl size=13861 sha256=3f65d42f48161118e9643e95c8b33dea8c39449e1cd348883fdd62de20f13700
  Stored in directory: /tmp/pip-ephem-wheel-cache-04dkxysp/wheels/8f/2d/45/d6856ebec9610a653a6f66783921cefb2b908fa4ee04453249
Successfully built SAM-2
  Attempting uninstall: SAM-2
    Found existing installation: SAM-2 1.0
    Uninstalling SAM-2-1.0:
      Successfully uninstalled SAM-2-1.0
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
초기 씨딩 완료: players=3, ball=1, details= [(0, 'player', 0), (1, 'player'

propagate in video: 100%|██████████| 1174/1174 [10:26<00:00,  1.87it/s]

✅ saved: /content/drive/MyDrive/Little_kid_0912/result/third_output.mp4


In [ ]:
# ================================
# ✅ SAM2 패키지 경로 등록 & 설치
# ================================
import sys, os

# SAM2가 Drive에 저장돼 있다고 가정
SAM2_PATH = "/content/drive/MyDrive/sam2"
sys.path.append(SAM2_PATH)

# 패키지 설치 (Colab 런타임마다 필요)
!pip install -e /content/drive/MyDrive/sam2

# ================================
# ✅ 라이브러리 import
# ================================
import torch, os, cv2, numpy as np
from ultralytics import YOLO
from sam2.build_sam import build_sam2_video_predictor

# ------------------------
# 준비
# ------------------------
def best_dtype():
    return torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16
dtype = best_dtype()

os.chdir("/content/drive/MyDrive/sam2")
ckpt = "/content/drive/MyDrive/sam2/checkpoints/sam2.1_hiera_large.pt"
cfg  = "configs/sam2.1/sam2.1_hiera_l.yaml"
pred = build_sam2_video_predictor(cfg, ckpt, vos_optimized=False)

MP4 = "/content/drive/MyDrive/Little_kid_0912/Video_data/0903_day1.mp4"
with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
    state = pred.init_state(MP4)

PLAYER_MODEL = "/content/drive/MyDrive/Little_kid_0912/pose/player/player_detect/weights/best.pt"
BALL_MODEL   = "/content/drive/MyDrive/Little_kid_0912/pose/ball/ball_800/train/weights/best.pt"
player_model = YOLO(PLAYER_MODEL)
ball_model   = YOLO(BALL_MODEL)

# ------------------------
# Util 함수
# ------------------------
def iou(boxA, boxB):
    # box: [x1,y1,x2,y2]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    inter = max(0, xB-xA+1) * max(0, yB-yA+1)
    areaA = (boxA[2]-boxA[0]+1)*(boxA[3]-boxA[1]+1)
    areaB = (boxB[2]-boxB[0]+1)*(boxB[3]-boxB[1]+1)
    return inter / float(areaA+areaB-inter+1e-6)

id2color, ema_masks, class_of = {}, {}, {}
def color_for(oid: int):
    if oid not in id2color:
        rng = np.random.default_rng(oid + 12345)
        id2color[oid] = tuple(int(c) for c in rng.integers(60,255,size=3))
    return id2color[oid]

# ------------------------
# 초기 씨딩 (각 객체가 '처음' 보이는 프레임에서)
# ------------------------
dev = torch.device("cuda")
next_oid = 0
class_of = {}   # {oid: "player"|"ball"}
found_players = False
found_ball = False

# 스캔 설정
SCAN_MAX = 300          # 처음 N프레임까지만 스캔 (원하면 크게)
STEP     = 1            # 1프레임 단위로 스캔 (속도 필요시 2~3으로)
CONF_P   = 0.30         # 선수 conf
CONF_B   = 0.05         # 공 conf(작아서 낮춤)
IMGSZ_B  = 1280         # 공 입력 해상도 상향(감지력↑)

cap = cv2.VideoCapture(MP4)

def seed_object(fi, xyxy, label, use_box=True):
    """
    xyxy=[x1,y1,x2,y2]를 받아 SAM2에 씨딩.
    - use_box=True: box 프롬프트 + 중심점(point) 함께 전달(권장)
    - use_box=False: 중심점(point)만 전달
    """
    global next_oid
    x1, y1, x2, y2 = xyxy
    oid = next_oid

    with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
        if use_box:
            # ✅ 박스 + 포인트(같은 CUDA)
            box_t = torch.tensor([[x1, y1, x2, y2]], device=dev, dtype=torch.float32)
            xc, yc = (x1 + x2) / 2.0, (y1 + y2) / 2.0
            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],        device=dev, dtype=torch.long)
            pred.add_new_points_or_box(
                state, fi, oid,
                points=pts, labels=labs,   # ← 포인트/라벨 명시
                box=box_t,                 # ← 박스도 함께
                normalize_coords=True
            )
        else:
            # 포인트만
            xc, yc = (x1 + x2) / 2.0, (y1 + y2) / 2.0
            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],        device=dev, dtype=torch.long)
            pred.add_new_points_or_box(
                state, fi, oid,
                points=pts, labels=labs,
                normalize_coords=True
            )

    class_of[oid] = label
    next_oid += 1
    return oid

seed_logs = []  # [(oid, label, fi)]

for fi in range(0, SCAN_MAX, STEP):
    ok, bgr = cap.read()
    if not ok:
        break

    # 초기 스캔 루프 내부 변경 부분만 발췌
    if not found_players:
        rp = player_model(bgr, conf=CONF_P, verbose=False)[0]
        if rp.boxes is not None and len(rp.boxes) > 0:
            for b in rp.boxes.xyxy.cpu().numpy().tolist():
                oid = seed_object(fi, b, "player", use_box=True)   # ✅ 박스 프롬프트
                seed_logs.append((oid, "player", fi))
            found_players = True

    if not found_ball:
        rb = ball_model(bgr, conf=CONF_B, imgsz=IMGSZ_B, verbose=False)[0]
        if rb.boxes is not None and len(rb.boxes) > 0:
            idx = int(rb.boxes.conf.argmax().cpu())
            b = rb.boxes.xyxy[idx].cpu().numpy().tolist()
            oid = seed_object(fi, b, "ball", use_box=True)         # ✅ 박스 프롬프트
            seed_logs.append((oid, "ball", fi))
            found_ball = True

    # 둘 다 찾았으면 조기 종료
    if found_players and found_ball:
        break

cap.release()

# 요약 로그
n_players = sum(1 for _,lbl,_ in seed_logs if lbl=="player")
n_balls   = sum(1 for _,lbl,_ in seed_logs if lbl=="ball")
print("초기 씨딩 완료:",
      f"players={n_players}, ball={n_balls},",
      "details=", seed_logs)

Obtaining file:///content/drive/MyDrive/sam2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=sam_2-1.0-0.editable-cp312-cp312-linux_x86_64.whl size=13861 sha256=232de273382fa2cf457ab9afc946e0d761a2b5f497d57a392acb70b5c04ce4ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-fa8esrbi/wheels/8f/2d/45/d6856ebec9610a653a6f66783921cefb2b908fa4ee04453249
Successfully built SAM-2
  Attempting uninstall: SAM-2
    Found existing installation: SAM-2 1.0
    Uninstalling SAM-2-1.0:
      Successfully uninstalled SAM-2-1.0
초기 씨딩 완료: players=3, ball=1, details= [(0, 'player', 0), (1, 'player', 0), (2, 'player', 0), (3, 'ball', 0)]


In [ ]:
def seed_from_yolo(res, label, fi=0):
    global next_oid
    added = []
    if res.boxes is not None and len(res.boxes)>0:
        for b in res.boxes.xyxy.cpu().numpy().tolist():
            x1,y1,x2,y2 = b
            xc,yc = (x1+x2)/2,(y1+y2)/2
            pts  = torch.tensor([[xc,yc]], device=dev, dtype=torch.float32)
            labs = torch.tensor([1],      device=dev, dtype=torch.long)
            oid  = next_oid
            with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
                pred.add_new_points_or_box(
                    state, fi, oid,
                    points=pts, labels=labs,
                    normalize_coords=True
                )
            class_of[oid] = label
            added.append((oid,[x1,y1,x2,y2]))
            next_oid += 1
    return added

# ------------------------
# 비디오 루프
# ------------------------
OUT_PATH = "/content/drive/MyDrive/Little_kid_0912/result/sam2_players_balls_reseed.mp4"
cap = cv2.VideoCapture(MP4)
W,H,fps = int(cap.get(3)),int(cap.get(4)),cap.get(5) or 30
out = cv2.VideoWriter(OUT_PATH, cv2.VideoWriter_fourcc(*"mp4v"), fps,(W,H))

reseed_interval = 15  # N프레임마다 재씨딩

with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
    for fi, obj_ids, masks in pred.propagate_in_video(state):
        ok, bgr = cap.read()
        if not ok: break
        overlay = bgr.copy()

        # ---------------- reseed ----------------
        if fi % reseed_interval == 0:
            res_p = player_model(bgr, conf=0.3, verbose=False)[0]
            res_b = ball_model(bgr,   conf=0.15, verbose=False)[0]

            for res, label in [(res_p, "player"), (res_b, "ball")]:
                if res.boxes is None:
                    continue

                for b in res.boxes.xyxy.cpu().numpy().tolist():
                    matched = False

                    # 이전 EMA 마스크에서 박스 만들고 IOU로 동일 객체 판단
                    for oid in list(class_of.keys()):
                        if class_of[oid] != label:
                            continue

                        prev_mask = ema_masks.get(oid)
                        if prev_mask is None:
                            continue

                        ys, xs = np.where(prev_mask > 0.5)
                        if xs.size == 0:
                            continue

                        box_prev = [xs.min(), ys.min(), xs.max(), ys.max()]
                        # (교체) reseed 블록의 "matched == True" 분기
                        if iou(box_prev, b) > 0.5:
                            # ✅ 같은 객체로 판단 → 박스 + 포인트 프롬프트로 보강 (모두 CUDA)
                            x1, y1, x2, y2 = b
                            box_t = torch.tensor([[x1, y1, x2, y2]], device=dev, dtype=torch.float32)

                            xc, yc = (x1 + x2) / 2.0, (y1 + y2) / 2.0
                            pts  = torch.tensor([[xc, yc]], device=dev, dtype=torch.float32)  # CUDA
                            labs = torch.tensor([1],        device=dev, dtype=torch.long)     # CUDA

                            with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
                                pred.add_new_points_or_box(
                                    state, fi, oid,
                                    points=pts, labels=labs,    # ← 꼭 함께 전달!
                                    box=box_t,                  # ← 박스 프롬프트
                                    normalize_coords=True
                                )
                            matched = True
                            break

                    if not matched:
                        # ✅ 매칭 실패 → 해당 박스 b만 새 oid로 씨딩
                        new_oid = seed_object(fi, b, label, use_box=True)  # ✅ 박스 프롬프트
                        seed_logs.append((new_oid, label, fi))

        # ---------------- 마스크 시각화 ----------------
        for oid,m in zip(obj_ids,masks):
            oid=int(oid)
            m=m.detach().cpu().numpy().squeeze()
            if m.ndim!=2: continue
            m=(m>0).astype(np.uint8)*255
            m=cv2.morphologyEx(m,cv2.MORPH_OPEN,np.ones((3,3),np.uint8))
            m=cv2.morphologyEx(m,cv2.MORPH_CLOSE,np.ones((3,3),np.uint8))
            m=cv2.GaussianBlur(m,(3,3),0)
            cur=(m>127).astype(np.float32)
            prev=ema_masks.get(oid)
            keep=0.5 if class_of.get(oid)=="ball" else 0.7
            ema=cur if prev is None else keep*prev+(1-keep)*cur
            ema_masks[oid]=ema; m_bin=ema>0.5

            col=(0,0,255) if class_of.get(oid)=="ball" else color_for(oid)
            overlay[m_bin]=col
            ys,xs=np.where(m_bin)
            if xs.size:
                x1,x2,y1,y2=xs.min(),xs.max(),ys.min(),ys.max()
                cv2.rectangle(overlay,(x1,y1),(x2,y2),col,2)
                cv2.putText(overlay,f"{class_of.get(oid)} {oid}",(x1,max(0,y1-6)),
                            cv2.FONT_HERSHEY_SIMPLEX,0.6,col,2)

        bgr=cv2.addWeighted(overlay,0.35,bgr,0.65,0)
        out.write(bgr)

cap.release(); out.release()
print("✅ saved:", OUT_PATH)

propagate in video: 100%|██████████| 405/405 [06:19<00:00,  1.07it/s]

✅ saved: /content/drive/MyDrive/Little_kid_0912/result/sam2_players_balls_reseed.mp4
